In [1]:
!pip install datasets
!pip install sacrebleu
!pip install rouge_score
from datasets import load_metric
from datasets import load_dataset
import pandas as pd

blue_metric = load_metric("sacrebleu")
rouge_metric = load_metric("rouge")
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 29.8 MB/s 
     |████████████████████████████████| 212 kB 73.0 MB/s 
     |████████████████████████████████| 101 kB 13.4 MB/s 
     |████████████████████████████████| 115 kB 67.9 MB/s 
     |████████████████████████████████| 127 kB 55.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 116 kB 29.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=783324647f1373eea9f63200895e804de1be228b02fca9994a95e8446a41f50a
  Stored in directory: /root/.cache/

In [2]:
dataset = load_dataset("json", data_files={'train':'train.json', 'test': 'test.json', 'validation':'validate.json'})
split_lengths = [len(dataset[split])for split in dataset]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-3c86ac76df642d45/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset['train'].column_names}")
print("\nDialogue:")
print(dataset["test"][0]["plain_text"])
print("\nSummary:")
print(dataset["test"][0]["summary"])


Split lengths: [540, 270, 90]
Features: ['plain_text', 'summary']

Dialogue:
The Service is only available to individuals aged 13 years or older Apple reserves the right at any time to modify this Agreement and to impose new or additional terms or conditions on your use of the Service, provided that Apple will give you 30 days’ advance notice of any material adverse change to the Service or applicable terms of service, unless it would not be reasonable to do so due to circumstances arising from legal, regulatory, or governmental action. Apple and its partners and licensors must collect, use, transmit, process and maintain your location data, including but not limited to the geographic location of your device and information related to your iCloud account (“Account”) and any devices registered thereunder, including but not limited to your Apple ID, device ID and name, and device type You may withdraw consent to Apple and its partners’ and licensors’ collection, use, transmission, proces

In [4]:
from tqdm import tqdm
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

def chunks(list_of_elements, batch_size):
    """Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]


def evaluate_summaries_pegasus(dataset, metric, model, tokenizer, batch_size=1, device=device, column_text="plain_text", column_summary="summary"):
    article_batches = list(chunks(dataset[column_text], batch_size))
    target_batches = list(chunks(dataset[column_summary], batch_size))
    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):
        inputs = tokenizer(article_batch, max_length=1024, truncation=True, padding="max_length", return_tensors="pt")
        print(inputs)
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                        attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                 clean_up_tokenization_spaces=True) for s in summaries]
        decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]
        metric.add_batch(predictions=decoded_summaries, references=target_batch)
    
    score = metric.compute()
    return score

In [5]:
!pip install transformers
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

test_sampled = dataset["test"].shuffle(seed=42).select(range(10))


model_ckpt = 'facebook/bart-large-cnn'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 21.5 MB/s 
     |████████████████████████████████| 6.6 MB 53.9 MB/s 


In [6]:

def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch["plain_text"], max_length=1024, truncation=True)
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch["summary"], max_length=128, truncation=True)
    return {"input_ids": input_encodings["input_ids"], "attention_mask": input_encodings["attention_mask"], 
            "labels": target_encodings["input_ids"]}

dataset_samsum_pt = dataset.map(convert_examples_to_features, batched=True)
columns = ["input_ids", "labels", "attention_mask"]
dataset_samsum_pt.set_format(type="torch", columns=columns)


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [7]:
from transformers import DataCollatorForSeq2Seq
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [8]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
 output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
 per_device_train_batch_size=1, per_device_eval_batch_size=1,
 weight_decay=0.01, logging_steps=1, push_to_hub=False,
 evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
 gradient_accumulation_steps=16)


In [9]:
trainer = Trainer(model=model, args=training_args,
 tokenizer=tokenizer, data_collator=seq2seq_data_collator,
 train_dataset=dataset_samsum_pt["train"],
 eval_dataset=dataset_samsum_pt["validation"])

In [11]:
trainer.train()
score = evaluate_summaries_pegasus(
    dataset["test"], rouge_metric, trainer.model, tokenizer,
    batch_size=1, column_text="plain_text", column_summary="summary")
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=[f"pegasus"])


The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, plain_text. If summary, plain_text are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 540
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 16
  Total optimization steps = 33


Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


  0%|          | 0/270 [00:00<?, ?it/s]

{'input_ids': tensor([[   0,  133, 1841,  ...,    1,    1,    1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])}


  0%|          | 1/270 [00:02<10:33,  2.36s/it]

...

{'input_ids': tensor([[   0, 3972,  104,  ...,    1,    1,    1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])}


100%|█████████▉| 269/270 [08:29<00:01,  1.75s/it]

{'input_ids': tensor([[   0, 1106,  370,  ..., 1122,  518,    2]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}


100%|██████████| 270/270 [08:31<00:00,  1.89s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.345268,0.166581,0.220032,0.21998


In [14]:
model.save_pretrained("content/model")

Configuration saved in content/model/config.json
Model weights saved in content/model/pytorch_model.bin


In [18]:
from transformers import AutoConfig, AutoTokenizer, AutoModel
custom_model = AutoModel.from_pretrained("content/model")

loading configuration file content/model/config.json
Model config BartConfig {
  "_name_or_path": "content/model",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "force_bos_token_to_be_generated": true,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  

In [23]:
from transformers import pipeline
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from datasets import load_dataset

summaries = {}
dataset = load_dataset("json", data_files={'test': 'test.json'})
sample_text = dataset["test"][1]["plain_text"]

pipe = pipeline("summarization", model ="content/model", tokenizer="facebook/bart-large-cnn", device=0)
pipe_out = pipe(sample_text)
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-06225b33af32af1e/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

loading configuration file content/model/config.json
Model config BartConfig {
  "_name_or_path": "content/model",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "force_bos_token_to_be_generated": true,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  

In [24]:
print(summaries)

{'bart': "You can opt out of being on the service's mailing list.\nYour personal data may be shared with third parties.\nThe service may use tracking pixels, web beacons, browser fingerprinting, and/or device fingerprinting on users.\nYou can delete your personal data if you do not want it used for any purpose.\nYou may be tracked by third parties, including third parties acting on your behalf.\nYour data is only collected for a limited period of time, unless there is a need for the service.\nYour information may be used for marketing purposes.\nYou have the right to request a copy of your data if it is misused or inaccurate."}
